### PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645-675) underperformed compared to schools with smaller budgets (<\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (95% passing vs 75%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [65]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [12]:
# Calculate the 8 requested quantities
total_schools = school_data_complete['School ID'].nunique()
total_students = school_data_complete['Student ID'].nunique()
total_budget = school_data_complete.groupby("School ID").budget.first().sum()
average_math_score = school_data_complete.math_score.mean()
average_reading_score = school_data_complete.reading_score.mean()
overall_passing_rate = 0.5 * average_math_score + 0.5 * average_reading_score
percent_passing_math = school_data_complete.math_score.loc[school_data_complete['math_score'] >= 70].count() / total_students * 100
percent_passing_reading = school_data_complete.reading_score.loc[school_data_complete['reading_score'] >= 70].count() / total_students * 100
# Convert to DataFrame 
summary_table = pd.DataFrame({'Total Schools':total_schools,'Total Students':total_students,'Total Budget':total_budget\
                             ,'Average Math Score':average_math_score, 'Average Reading Score':average_reading_score\
                             ,'% Passing Math':percent_passing_math,'% Passing Reading':percent_passing_reading\
                             ,"% Overall Passing Rate":overall_passing_rate},index=np.arange(1))
# Clean up formatting and display
summary_table['Total Students'] = summary_table['Total Students'].map('{:,}'.format)
summary_table['Total Budget'] = summary_table['Total Budget'].map('${:,}'.format)
summary_table.style.hide_index()

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
15,"39,170","$24,649,428",78.9854,81.8778,74.9809,85.8055,80.4316


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
# Use a bulk groupby("school_name") to get name, type, total students, total budget, average math score, average reading score
school_names1_df = school_data_complete.groupby('school_name').agg({'type':'first','Student ID':'nunique','budget':'first'\
                                                              ,'math_score':'mean','reading_score':'mean'}).reset_index()
# Fix the multi-index, the column names will be 'type','Student ID','budget','math_score','reading_score'
school_names1_df.columns = school_names1_df.columns.get_level_values(0)
# Add a column for budget per pupil
school_names1_df['Budget per Student'] = school_names1_df['budget'] / school_names1_df['Student ID']
# Create new DataFrames with groupby("school_name") for percent passing math and reading
# This is done by first creating two seprate DataFrames that holds the counts of passing students ...
school_names_mp1_df = school_data_complete[school_data_complete['math_score'] > 70].groupby("school_name")[['math_score']].count().reset_index() 
school_names_rp1_df = school_data_complete[school_data_complete['reading_score'] > 70].groupby("school_name")[['reading_score']].count().reset_index()
# Then adding columns to capture the percentage passing to each data frame
school_names_mp1_df['% Passing Math'] = school_names_mp1_df['math_score'] / school_names1_df['Student ID'] * 100
school_names_rp1_df['% Passing Reading'] = school_names_rp1_df['reading_score'] / school_names1_df['Student ID'] * 100
# Next, these new columns will be merged into a single DataFrame
school_names_merge_mr = school_names_mp1_df.merge(school_names_rp1_df,on='school_name',how='outer')
# Add the final column for the overall passing rate
school_names_merge_mr['Overall Passing Rate'] = 0.5 * (school_names_merge_mr['% Passing Math'] + school_names_merge_mr['% Passing Reading'])
# Then trimming away the columns we no longer need to use & preparing to merge with school_names1_df
school_names2_df = school_names_merge_mr.drop(['math_score','reading_score'], axis=1)
# Merge DataFrames school_names1_df and school_names2_df to create the final table
school_names1_df = school_names1_df.merge(school_names2_df,on='school_name',how='outer')
# Lastly, clean up the column names to be more meaningful and fix formatting
school_names1_df = school_names1_df.rename(columns = {'type':'School Type','Student ID':'Total Students','budget':'Total Budget', \
                                    'math_score':'Average Math Score','reading_score':'Average Reading Score','school_name':'Name'})
school_names1_df['Total Students'] = school_names1_df['Total Students'].map('{:,}'.format)
school_names1_df['Total Budget'] = school_names1_df['Total Budget'].map('${:,}'.format)
school_names1_df['Budget per Student'] = school_names1_df['Budget per Student'].map('${:,.2f}'.format)
school_names1_df.style.hide_index()

Name,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Budget per Student,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,"4,976","$3,124,928",77.0484,81.034,$628.00,64.6302,79.3006,71.9654
Cabrera High School,Charter,"1,858","$1,081,356",83.0619,83.9758,$582.00,89.5587,93.8644,91.7115
Figueroa High School,District,"2,949","$1,884,411",76.7118,81.158,$639.00,63.7504,78.4334,71.0919
Ford High School,District,"2,739","$1,763,916",77.1026,80.7463,$644.00,65.7539,77.51,71.632
Griffin High School,Charter,"1,468","$917,500",83.3515,83.8168,$625.00,89.7139,93.3924,91.5531
Hernandez High School,District,"4,635","$3,022,020",77.2898,80.9344,$652.00,64.7465,78.1877,71.4671
Holden High School,Charter,427,"$248,087",83.8033,83.815,$581.00,90.6323,92.74,91.6862
Huang High School,District,"2,917","$1,910,635",76.6294,81.1827,$655.00,63.3185,78.8138,71.0662
Johnson High School,District,"4,761","$3,094,650",77.0725,80.9664,$650.00,63.8521,78.2819,71.067
Pena High School,Charter,962,"$585,858",83.8399,84.0447,$609.00,91.684,92.2037,91.9439


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
school_names1_df.sort_values('Overall Passing Rate',ascending=False).head(5).style.hide_index()

Name,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Budget per Student,% Passing Math,% Passing Reading,Overall Passing Rate
Wilson High School,Charter,"2,283","$1,319,574",83.2742,83.9895,$578.00,90.933,93.2545,92.0937
Pena High School,Charter,962,"$585,858",83.8399,84.0447,$609.00,91.684,92.2037,91.9439
Wright High School,Charter,"1,800","$1,049,400",83.6822,83.955,$583.00,90.2778,93.4444,91.8611
Cabrera High School,Charter,"1,858","$1,081,356",83.0619,83.9758,$582.00,89.5587,93.8644,91.7115
Holden High School,Charter,427,"$248,087",83.8033,83.815,$581.00,90.6323,92.74,91.6862


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
school_names1_df.sort_values('Overall Passing Rate').head(5).style.hide_index()

Name,School Type,Total Students,Total Budget,Average Math Score,Average Reading Score,Budget per Student,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",76.8427,80.7447,$637.00,64.066,77.7444,70.9052
Huang High School,District,"2,917","$1,910,635",76.6294,81.1827,$655.00,63.3185,78.8138,71.0662
Johnson High School,District,"4,761","$3,094,650",77.0725,80.9664,$650.00,63.8521,78.2819,71.067
Figueroa High School,District,"2,949","$1,884,411",76.7118,81.158,$639.00,63.7504,78.4334,71.0919
Hernandez High School,District,"4,635","$3,022,020",77.2898,80.9344,$652.00,64.7465,78.1877,71.4671


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# Create series for each grade ...
# Make the first series a DataFrame to get started
math_score_grades = school_data_complete[school_data_complete['grade'] == '9th'].groupby("school_name")[['math_score']].mean().reset_index()
# Make the others into lists
math_score_10 = school_data_complete[school_data_complete['grade'] == '10th'].groupby("school_name").math_score.mean().tolist()
math_score_11 = school_data_complete[school_data_complete['grade'] == '9th'].groupby("school_name").math_score.mean().tolist()
math_score_12 = school_data_complete[school_data_complete['grade'] == '12th'].groupby("school_name").math_score.mean().tolist()
# Then add them to the DataFrame
math_score_grades['10th'] = math_score_10
math_score_grades['11th'] = math_score_11
math_score_grades['12th'] = math_score_12
# Lastly clean up the column names 
math_score_grades = math_score_grades.rename(columns = {'school_name':'School Name','math_score':'9th'})
# Display without index
math_score_grades.style.hide_index()

School Name,9th,10th,11th,12th
Bailey High School,77.0837,76.9968,77.0837,76.4922
Cabrera High School,83.0947,83.1545,83.0947,83.2775
Figueroa High School,76.403,76.54,76.403,77.1514
Ford High School,77.3613,77.6723,77.3613,76.18
Griffin High School,82.044,84.2291,82.044,83.3562
Hernandez High School,77.4385,77.3374,77.4385,77.1866
Holden High School,83.7874,83.4298,83.7874,82.8554
Huang High School,77.0273,75.9087,77.0273,77.2256
Johnson High School,77.1879,76.6911,77.1879,76.8632
Pena High School,83.6255,83.372,83.6255,84.1215


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# Create series for each grade ...
# Make the first series a DataFrame to get started
reading_score_grades = school_data_complete[school_data_complete['grade'] == '9th'].groupby("school_name")[['reading_score']].mean().reset_index()
# Make the others into lists
reading_score_10 = school_data_complete[school_data_complete['grade'] == '10th'].groupby("school_name").reading_score.mean().tolist()
reading_score_11 = school_data_complete[school_data_complete['grade'] == '9th'].groupby("school_name").reading_score.mean().tolist()
reading_score_12 = school_data_complete[school_data_complete['grade'] == '12th'].groupby("school_name").reading_score.mean().tolist()
# Then add them to the DataFrame
reading_score_grades['10th'] = reading_score_10
reading_score_grades['11th'] = reading_score_11
reading_score_grades['12th'] = reading_score_12
# Lastly clean up the column names 
reading_score_grades
reading_score_grades = reading_score_grades.rename(columns = {'school_name':'School Name','reading_score':'9th'})
# Display without index
reading_score_grades.style.hide_index()

School Name,9th,10th,11th,12th
Bailey High School,81.3032,80.9072,81.3032,80.9125
Cabrera High School,83.6761,84.2532,83.6761,84.288
Figueroa High School,81.1986,81.4089,81.1986,81.3849
Ford High School,80.6327,81.2627,80.6327,80.6623
Griffin High School,83.3692,83.7069,83.3692,84.0137
Hernandez High School,80.8669,80.6601,80.8669,80.8571
Holden High School,83.6772,83.3246,83.6772,84.6988
Huang High School,81.2903,81.5124,81.2903,80.306
Johnson High School,81.2607,80.7734,81.2607,81.2276
Pena High School,83.8073,83.612,83.8073,84.5912


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [58]:
# Sample bins and labels
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
# In order to compare average scores based on total students (i.e. weighted for school size), rather than just 
# an "average of average school scores", budget per student classification needs to happen at the "student-data" level,
# so the original student-level DataFrame needs new series for budget and spending range per student.
school_data_complete['Budget per Student'] = school_data_complete['budget'] / school_data_complete['size']
school_data_complete['Spending Range per Student'] = pd.cut(school_data_complete['Budget per Student'], spending_bins, labels=group_names)                                                                  
# Now, to build the table, start with a DataFrame that captures the count of students in each group, along with their average math and reading scores
scores_by_spending = school_data_complete.groupby('Spending Range per Student').agg({'math_score':['count','mean'],'reading_score':'mean'}).reset_index()
# Fix the column names to make things easier to handle
scores_by_spending.columns = ['Spending Range per Student','Student Count','Average Math Score', 'Average Reading Score']
# To add the percentage passing math, first generate counts of passing students by category, then add them to the table
passing_math_list = school_data_complete[school_data_complete['math_score'] >= 70].groupby('Spending Range per Student')['math_score'].count().tolist()
passing_reading_list = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('Spending Range per Student')['reading_score'].count().tolist()
scores_by_spending['Number Passing Math'] = passing_math_list
scores_by_spending['Number Passing Reading'] = passing_reading_list
# Make new columns with the percentages passing generated from the totals
scores_by_spending['% Passing Math'] = 100 * scores_by_spending['Number Passing Math'] / scores_by_spending['Student Count']
scores_by_spending['% Passing Reading'] = 100 * scores_by_spending['Number Passing Reading'] / scores_by_spending['Student Count']
scores_by_spending['Overall Passing Rate'] = 0.5 * scores_by_spending['% Passing Math'] + 0.5 * scores_by_spending['% Passing Reading']
# Lastly, trim out the columns not requested
scores_by_spending = scores_by_spending.drop(['Student Count','Number Passing Math','Number Passing Reading'], axis=1)
scores_by_spending.style.hide_index()

Spending Range per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
<$585,83.3631,83.964,93.7029,96.6866,95.1947
$585-615,83.5292,83.8384,94.1241,95.8869,95.0055
$615-645,78.0616,81.4341,71.4004,83.6148,77.5076
$645-675,77.0493,81.0056,66.2308,81.1094,73.6701


## Scores by School Size

* Perform the same operations as above, based on school size.

In [62]:
# Sample bins and labels
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
# Averages will be computed at the student level, not simply an average of school scores.
# The latter would overweight schools at the small end of the bins.
school_data_complete['Size Range'] = pd.cut(school_data_complete['size'], size_bins, labels=group_names)                                                                  
# Now, to build the table, start with a DataFrame that captures the count of students in each group, along with their average math and reading scores
scores_by_size = school_data_complete.groupby('Size Range').agg({'math_score':['count','mean'],'reading_score':'mean'}).reset_index()
# Fix the column names to make things easier to handle
scores_by_size.columns = ['Size Range','Student Count','Average Math Score', 'Average Reading Score']
# To add the percentage passing math, first generate counts of passing students by category, then add them to the table
passing_math_list = school_data_complete[school_data_complete['math_score'] >= 70].groupby('Size Range')['math_score'].count().tolist()
passing_reading_list = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('Size Range')['reading_score'].count().tolist()
scores_by_size['Number Passing Math'] = passing_math_list
scores_by_size['Number Passing Reading'] = passing_reading_list
# Make new columns with the percentages passing generated from the totals
scores_by_size['% Passing Math'] = 100 * scores_by_size['Number Passing Math'] / scores_by_size['Student Count']
scores_by_size['% Passing Reading'] = 100 * scores_by_size['Number Passing Reading'] / scores_by_size['Student Count']
scores_by_size['Overall Passing Rate'] = 0.5 * scores_by_size['% Passing Math'] + 0.5 * scores_by_size['% Passing Reading']
# Lastly, trim out the columns not requested
scores_by_size = scores_by_size.drop(['Student Count','Number Passing Math','Number Passing Reading'], axis=1)
scores_by_size.style.hide_index()

Size Range,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Small (<1000),83.8287,83.9741,93.9525,96.0403,94.9964
Medium (1000-2000),83.3727,83.868,93.6165,96.7731,95.1948
Large (2000-5000),77.4776,81.1987,68.6524,82.1252,75.3888


## Scores by School Type

* Perform the same operations as above, based on school type.

In [64]:
# To build the table, start with a DataFrame that captures the count of students in each group, along with their average math and reading scores
scores_by_type = school_data_complete.groupby('type').agg({'math_score':['count','mean'],'reading_score':'mean'}).reset_index()
# Fix the column names to make things easier to handle
scores_by_type.columns = ['School Type','Student Count','Average Math Score', 'Average Reading Score']
# To add the percentage passing math, first generate counts of passing students by category, then add them to the table
passing_math_list = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['math_score'].count().tolist()
passing_reading_list = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['reading_score'].count().tolist()
scores_by_type['Number Passing Math'] = passing_math_list
scores_by_type['Number Passing Reading'] = passing_reading_list
# Make new columns with the percentages passing generated from the totals
scores_by_type['% Passing Math'] = 100 * scores_by_type['Number Passing Math'] / scores_by_type['Student Count']
scores_by_type['% Passing Reading'] = 100 * scores_by_type['Number Passing Reading'] / scores_by_type['Student Count']
scores_by_type['Overall Passing Rate'] = 0.5 * scores_by_type['% Passing Math'] + 0.5 * scores_by_type['% Passing Reading']
# Lastly, trim out the columns not requested
scores_by_type = scores_by_type.drop(['Student Count','Number Passing Math','Number Passing Reading'], axis=1)
scores_by_type.style.hide_index()

School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Charter,83.4062,83.9028,93.7018,96.6459,95.1739
District,76.987,80.9625,66.5184,80.9052,73.7118
